In [145]:
import pandas as pd
import numpy as np
import os
import json

In [146]:
labels = ['Right_PCS', 'Age']
targets = ['test_values.json', 'train_val_values.json']
embeddings = ['ACC_custom_embeddings', 'UKB_5percent_embeddings']
metrics = ['test_auc', 'train_val_auc']

output_dir = '/volatile/jl277509/Runs/02_STS_babies/Program/Output'

In [158]:
df_list=[]
for label, target, embedding, metric in zip(labels, targets, embeddings, metrics):
    print(label, target, embedding)
    selected_models_dirs = [root for root, dirs, files in os.walk(output_dir, followlinks=True)
                            if (embedding in root) & (root.endswith(label)) & (target in files) & ('best_model' not in root)]
    value_list=[]
    for model in selected_models_dirs:
        with open(os.path.join(model,target), 'r') as f:
            perf = json.load(f)
        if isinstance(perf[metric], float):
            value=perf[metric]
        else:
            value = perf[metric][0]
        value_list.append(value)

    models_dirs = [elem.split('/')[-3][:-2] for elem in selected_models_dirs]
    
    df = pd.DataFrame({'model': models_dirs, label: value_list})
    df_list.append(df)

Right_PCS test_values.json ACC_custom_embeddings
Age train_val_values.json UKB_5percent_embeddings


In [163]:
models_table = ['random_all_keep_bottom','random_no_keep_no_cutin', 'random_no_keep_no_cutout',
                'random_no_keep_no_translation_only', 'random_no_keep_no_trimdepth', 'translation_only_no_noise']

In [232]:
df.loc[df['model']=='random_all_keep_bottom']

,model,Age
129,random_all_keep_bottom,0.5107
153,random_all_keep_bottom,0.5115
161,random_all_keep_bottom,0.5113
175,random_all_keep_bottom,0.5289
184,random_all_keep_bottom,0.5030
223,random_all_keep_bottom,0.5411
224,random_all_keep_bottom,0.5239
227,random_all_keep_bottom,0.5107
228,random_all_keep_bottom,0.4930
236,random_all_keep_bottom,0.5115


In [249]:
# NB: certains modèles y sont plusieurs fois... C'est ok s'ils sont tous en le même nombre d'exemplaires
values = {label: [] for label in labels}
for name in models_table:
    print(name)
    for df, label in zip(df_list, labels):
        print(label)
        l_val = df.loc[df['model']==name][label].tolist()[:5]
        print(f'nb models : {len(l_val)}')
        print(f'{name}: {np.mean(l_val):.3f}, {np.std(l_val):.3f}')
        values[label].append(l_val) # choose list, mean or std
# if list...
values['criterion (Right_PCS - Age)'] = [[f1-f2 for f1, f2 in zip(e1,e2)] for e1, e2 in zip(values['Right_PCS'],values['Age'])]

random_all_keep_bottom
Right_PCS
nb models : 5
random_all_keep_bottom: 0.871, 0.005
Age
nb models : 5
random_all_keep_bottom: 0.513, 0.009
random_no_keep_no_cutin
Right_PCS
nb models : 5
random_no_keep_no_cutin: 0.862, 0.007
Age
nb models : 5
random_no_keep_no_cutin: 0.553, 0.005
random_no_keep_no_cutout
Right_PCS
nb models : 5
random_no_keep_no_cutout: 0.879, 0.005
Age
nb models : 5
random_no_keep_no_cutout: 0.539, 0.016
random_no_keep_no_translation_only
Right_PCS
nb models : 5
random_no_keep_no_translation_only: 0.867, 0.007
Age
nb models : 5
random_no_keep_no_translation_only: 0.561, 0.005
random_no_keep_no_trimdepth
Right_PCS
nb models : 5
random_no_keep_no_trimdepth: 0.861, 0.008
Age
nb models : 5
random_no_keep_no_trimdepth: 0.559, 0.003
translation_only_no_noise
Right_PCS
nb models : 5
translation_only_no_noise: 0.837, 0.016
Age
nb models : 5
translation_only_no_noise: 0.519, 0.004


In [250]:
df_values = pd.DataFrame(values)
df_values['model']=models_table
#df_values['criterion (Right_PCS - Age)'] = df_values['Right_PCS'].to_numpy()-df_values['Age'].to_numpy()

In [251]:
values.keys()

dict_keys(['Right_PCS', 'Age', 'criterion (Right_PCS - Age)'])

In [252]:
for k, model in enumerate(models_table):
    d=df_values.loc[df_values['model']==models_table[k]]
    l = [e1-e2 for e1, e2 in zip(d['Right_PCS'].tolist()[0],d['Age'].tolist()[0])]
    print(f'{model}: {np.std(l)}')

random_all_keep_bottom: 0.009692475221440577
random_no_keep_no_cutin: 0.008104246897852031
random_no_keep_no_cutout: 0.013572995450152434
random_no_keep_no_translation_only: 0.005850965686409789
random_no_keep_no_trimdepth: 0.0093041697876062
translation_only_no_noise: 0.02038098480391109


In [239]:
values['Right_PCS'][0]

[0.8690154867256636,
 0.8799225663716814,
 0.8691924778761063,
 0.8689159292035399,
 0.8671515486725664,
 0.8604535398230089,
 0.8419690265486726,
 0.8690154867256636,
 0.8892975663716814,
 0.8799225663716814,
 0.8691924778761063,
 0.8797511061946901,
 0.8689159292035399,
 0.8775276548672567,
 0.8671515486725664,
 0.8690154867256636,
 0.8799225663716814,
 0.8691924778761063,
 0.8689159292035399,
 0.8671515486725664,
 0.8690154867256636,
 0.8799225663716814,
 0.8691924778761063,
 0.8689159292035399,
 0.8671515486725664,
 0.8604535398230089,
 0.8419690265486726,
 0.8892975663716814,
 0.8797511061946901,
 0.8775276548672567,
 0.8604535398230089,
 0.8419690265486726,
 0.8892975663716814,
 0.8797511061946901,
 0.8775276548672567,
 0.8797511061946901]

In [230]:
np.std([e1-e2 for e1,e2 in zip(values['Right_PCS'][0],values['Age'][0])][:5])

0.009692475221440577